In [ ]:
# Our API Code


In [1]:
import json
import os
from pathlib import Path
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd
import json
import pandas as pd

import json
import pandas as pd
import glob
import re

from pandas import DataFrame


class KabaddiDataAPI:
    def __init__(self):
        self.base_path = "./MatchData_pbp"
    # for a season - display the standings

    def internal_create_matches_list(self, team, group_name):
        matches = []
        for match in team['match_result']['match']:
            match_info = {
                'Group': group_name,
                'match_id': match['id'],
                'date': match['date'],
                'teama_id': match['teama_id'],
                'result': match['result'],
                'teama_short_name': match['teama_short_name'],
                'teama_score': match['teama_score'],
                'teamb_id': match['teamb_id'],
                'teamb_short_name': match['teamb_short_name'],
                'teamb_score': match['teamb_score'],
                'match_result': match['match_result']
            }

            print(match_info)
            matches.append(match_info)
        return matches

    def internal_process_matches(self, matches_list):
        matches_df = pd.DataFrame(matches_list)
        matches_df = matches_df[(matches_df['result'].isin(['W', 'T'])) | (matches_df['result'].isnull())]
        matches_df = matches_df.sort_values(by='date', ascending=True)
        matches_df = matches_df.set_index('match_id').rename_axis('match id')
        return matches_df





    # DEPRECIATED
    def get_pkl_standings_matches(self, season=None, qualified=False, team_id=None, matches=False):

        if season is None:
            season = 10

        file_path = Path(f"./PKL_Standings/pkl_standings_s{season}.json")

        with open(file_path, 'r') as f:
            data = json.load(f)

        standings = data['standings']

        team_standings_info_list, qualified_teams_info_df, matches_list, = [], [], []

        season_name = standings['series_name']
        champion_team_id = standings['champion_id']

        if len(standings['groups']) == 0:
            return pd.DataFrame()

        if len(standings['groups']) == 1:

            group = standings['groups'][0]
            if 'name' in group and group['name'] != "":
                group_name = group['name']
            else:
                group_name = 'Main'

            for team in group['teams']['team']:
                if team_id is None or int(team['team_id']) == team_id:
                    team_standings_info = {
                        'Group': group_name,
                        'Season': season,
                        'Team_Id': team['team_id'],
                        'Team_Name': team['team_name'],
                        # 'team_short_name': team['team_short_name'],
                        'League_position': team['position'],
                        'Matches_played': team['played'],
                        'Wins': team['wins'],
                        'Lost': team['lost'],
                        'Tied': team['tied'],
                        'Draws': team['draws'],
                        'No Result': team['noresult'],
                        'League_points': team['points'],
                        'Score_diff': team['score_diff'],
                        'Qualified': team['is_qualified'],
                    }

                    if qualified and team['is_qualified']:
                        qualified_teams_info_df.append(team_standings_info)

                    team_standings_info_list.append(team_standings_info)

                    if matches:
                        matches_list.extend(self.internal_create_matches_list(team, group_name))

        else:

            for group in standings['groups']:

                if 'name' in group and group['name'] != "":
                    group_name = group['name']
                else:
                    group_name = 'Main'

                for team in group['teams']['team']:
                    if team_id is None or int(team['team_id']) == team_id:
                        team_info = {
                            'Group': group_name,
                            'Season': season,
                            'Team_Id': team['team_id'],
                            'Team_Name': team['team_name'],
                            # 'team_short_name': team['team_short_name'],
                            'League_position': team['position'],
                            'Matches_played': team['played'],
                            'Wins': team['wins'],
                            'Lost': team['lost'],
                            'Tied': team['tied'],
                            'Draws': team['draws'],
                            'No Result': team['noresult'],
                            'League_points': team['points'],
                            'Score_diff': team['score_diff'],
                            'Qualified': team['is_qualified'],
                        }

                        if qualified and team['is_qualified']:
                            qualified_teams_info_df.append(team_info)

                        team_standings_info_list.append(team_info)

                        if matches:
                            matches_list.extend(self.internal_create_matches_list(team, group_name))

        team_info_df = pd.DataFrame(team_standings_info_list)

        if qualified:
            qualified_teams_df = pd.DataFrame(qualified_teams_info_df)
            return qualified_teams_df, team_info_df

        if matches:
            matches_df = self.internal_process_matches(matches_list)
            return team_info_df, matches_df

        else:
            return team_info_df


    def get_pkl_standings(self, season=None, qualified=False, team_id=None):
        
        if season is None:
            season = 10

        file_path = Path(f"./PKL_Standings/pkl_standings_s{season}.json")

        with open(file_path, 'r') as f:
            data = json.load(f)

        standings = data['standings']
        team_standings_info_list, qualified_teams_standings_info_list = [], []

        if len(standings['groups']) == 0:
            return pd.DataFrame()

        def process_team(group_name, team):
            team_info = {
                'Group': group_name,
                'Season': season,
                'Team_Id': team['team_id'],
                'Team_Name': team['team_name'],
                'League_position': team['position'],
                'Matches_played': team['played'],
                'Wins': team['wins'],
                'Lost': team['lost'],
                'Tied': team['tied'],
                'Draws': team['draws'],
                'No Result': team['noresult'],
                'League_points': team['points'],
                'Score_diff': team['score_diff'],
                'Qualified': team['is_qualified'],
            }

            if (team_id is None or int(team['team_id']) == team_id):
                team_standings_info_list.append(team_info)
                if qualified and team['is_qualified']:
                    qualified_teams_standings_info_list.append(team_info)

        for group in standings['groups']:
            if 'name' in group and group['name'] != "":
                group_name = group['name']
            else:
                group_name = 'Main'
            for team in group['teams']['team']:
                process_team(group_name, team)

        team_info_df = pd.DataFrame(team_standings_info_list)

        if qualified:
            qualified_teams_df = pd.DataFrame(qualified_teams_standings_info_list)
            return qualified_teams_df, team_info_df

        return team_info_df


    def get_season_matches(self, season="all"):
        
        matches_list = []

        # Determine the file(s) to load based on the season input
        if season == "all":
            files = glob.glob('./Matches-Overview/S*_PKL_MatchData.json')
            # Sort the files based on the season number
            files = sorted(files, key=lambda x: int(re.search(r'S(\d+)', x).group(1)))
            print(files)

        else:
            files = [f'./Matches-Overview/S{season}_PKL_MatchData.json']

        for file in files:
            
            with open(file) as f:
                data = json.load(f)


            for match in data['matches']:

                team_name, team_id, team_score = [], [], []

                for p in match['participants']:
                    # print(p['name'])
                    team_name.append(p['name'])
                    team_id.append(p['id'])
                    team_score.append(p['value'])


                match_details = {
                    "Season": match['tour_name'].split(",")[0].split(" ")[-1],
                    'Match_ID': match['game_id'],
                    "Match_Name": match['event_name'],
                    "League_Stage": match['event_stage'],
                    "Year": match['tour_name'].split(",")[1].strip(),
                    "Venue": match['venue_name'].lower().title().strip(),
                    'Match_Outcome': match['event_sub_status'],
                    "Start_Date": match['start_date'],
                    "End_Date": match['end_date'],
                    "Result": match['result_code'],
                    "Winning Margin": match['winning_margin'],
                    'team_score_1': team_score[0],
                    'team_score_2': team_score[1],
                    'team_name_1': team_name[0],
                    'team_id_1': team_id[0],
                    'team_name_2': team_name[1],
                    'team_id_2': team_id[1],
                }

                

                matches_list.append(match_details)

        # Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(matches_list)

        df.to_csv("matches_data.csv")

        # Display the DataFrame
        return df


    def get_team_info(self, team_id, season='overall'):

        if season != 'overall':
            season = int(season)
        
        df_team_aggregated_stats = pd.read_csv("./Team-Wise-Data/PKL_AggregatedTeamStats.csv")
        df_team_raider_skills = pd.read_csv("./Team-Wise-Data/ALL_Raider_Skills_Merged.csv")
        df_team_defender_skills = pd.read_csv("./Team-Wise-Data/ALL_Defensive_Skills_Merged.csv")

        team_id = int(team_id)

        def find_team_column(dataframe, team_id):
            for col in dataframe.columns:
                if f"({team_id})" in col:
                    return col
            return None
        team_column_team_raider_skills = find_team_column(df_team_raider_skills, team_id)
        team_column_team_defender_skills = find_team_column(df_team_defender_skills, team_id)
        
        

        if season == 'overall':

            filtered_team_aggregated_stats = df_team_aggregated_stats[df_team_aggregated_stats['team_id'] == team_id]
            
            rows_overall = filtered_team_aggregated_stats[filtered_team_aggregated_stats['season'] == 'all']
            other_rows = filtered_team_aggregated_stats[filtered_team_aggregated_stats['season'] != 'all']

            filtered_team_aggregated_stats = pd.concat([rows_overall, other_rows]).reset_index(drop=True)
            filtered_team_raider_skills = None
            filtered_team_defender_skills = None

        
        else:

            df_team_aggregated_stats['team_id'] = pd.to_numeric(df_team_aggregated_stats['team_id'], errors='coerce')
            df_team_aggregated_stats['season'] = pd.to_numeric(df_team_aggregated_stats['season'], errors='coerce')
            
            filtered_team_aggregated_stats = df_team_aggregated_stats[df_team_aggregated_stats['season'] == season]
            filtered_team_aggregated_stats = filtered_team_aggregated_stats[filtered_team_aggregated_stats['team_id'] == team_id]

            if team_column_team_raider_skills:            
                filtered_team_raider_skills = df_team_raider_skills[df_team_raider_skills['Season'] == season]
                filtered_team_raider_skills = filtered_team_raider_skills[['Season','Skill Type','Skill Name',team_column_team_raider_skills]].reset_index(drop=True)
            else:
                filtered_team_raider_skills = None


            if team_column_team_defender_skills:
                filtered_team_defender_skills = df_team_defender_skills[df_team_defender_skills['Season'] == season]
                filtered_team_defender_skills = filtered_team_defender_skills[['Season','Skill Type','Skill Name',team_column_team_defender_skills]].reset_index(drop=True)
            else:
                filtered_team_defender_skills = None

        if filtered_team_aggregated_stats.empty:
            print(f"No data found in CSV for team_id {team_id} in season {season}")
            return None, None, None, None, None
        

        rank_columns = [col for col in filtered_team_aggregated_stats.columns if col.endswith('_rank')]
        value_columns = [col for col in filtered_team_aggregated_stats.columns if col.endswith('_value')]
        per_match_columns = [col for col in filtered_team_aggregated_stats.columns if col.endswith('_per-match')]

        # print(f"len rank cols: {len(rank_columns)}")
        # print(f"len value cols: {len(value_columns)}")
        # print(f"len per match cols: {len(per_match_columns)}")

        df_rank = filtered_team_aggregated_stats[['season', 'team_id', 'team_name' , 'matches_played'] + rank_columns]
        df_value = filtered_team_aggregated_stats[['season', 'team_id', 'team_name' , 'matches_played'] + value_columns]
        df_per_match = filtered_team_aggregated_stats[['season', 'team_id', 'team_name' , 'matches_played'] + per_match_columns]


        if season == 'overall':
            return df_rank, df_value, df_per_match, filtered_team_raider_skills, filtered_team_defender_skills
        else:
            return df_rank.T, df_value.T, df_per_match.T, filtered_team_raider_skills, filtered_team_defender_skills
            


    def get_team_matches(self, season, team_id :str):

        # pass

        df = self.get_season_matches(season=season)

        team_id = str(team_id)
        df = df[(df['team_id_1'] == team_id) | (df['team_id_2'] == team_id)]


        # print(df.columns)
        return df

        # matches_list = []

        # # Determine the file(s) to load based on the season input
        # if season == "all":
        #     files = glob.glob('./Matches-Overview/S*_PKL_MatchData.json')
        # else:
        #     files = [f'./Matches-Overview/S{season}_PKL_MatchData.json']

        # for file in files:
        #     with open(file) as f:
        #         data = json.load(f)

        #     # Loop over each match in the file
        #     for match in data['matches']:
        #         if match['participants'][0]['id']==team_id or  match['participants'][1]['id']==team_id:
        #             match_details = {
        #                 "Match Name": match['event_name'],
        #                 'Match ID': match['game_id'],
        #                 "Tour Name": match['tour_name'],
        #                 "Venue": match['venue_name'],
        #                 'Match_Outcome': match['event_sub_status'],
        #                 "Date": match['start_date'],
        #                 "Result": match['event_sub_status'],
        #                 "Winning Margin": match['winning_margin']
        #             }
        #             matches_list.append(match_details)
        #     df = pd.DataFrame(matches_list)
        #     return df
    

    def build_team_roster(self, team_id, season):
        roster = {}
        team_id = int(team_id)
        team_name = ""
        total_matches = 0

        for folder_name in os.listdir("./MatchData_pbp"):
            if f"Season_{season}" in folder_name:
                directory_path = os.path.join("./MatchData_pbp", folder_name)
                break
        else:
            print(f"No data found for season {season}")
            return pd.DataFrame()

        for filename in os.listdir(directory_path):
            if filename.endswith(".json"):
                file_path = os.path.join(directory_path, filename)
                with open(file_path, 'r') as f:
                    match_data = json.load(f)

                if 'gameData' in match_data:
                    match_data = match_data['gameData']

                series_dict = {10: '44', 9: '25', 8: '20', 7: '11', 6: '10', 5: '8', 3: '3', 2: '2', 1: '1', 4: '4'}
                season_id = series_dict.get(int(season))

                if int(match_data['match_detail']['series']['id']) == int(season_id):
                    for team in match_data['teams']['team']:
                        if int(team['id']) == team_id:
                            total_matches += 1
                            team_name = team['name']
                            for player in team['squad']:
                                player_id = player['id']
                                if player_id not in roster:
                                    roster[player_id] = {
                                        'Player ID': player_id,
                                        'Name': player['name'],
                                        'Jersey Number': player.get('jersey'),
                                        'Captain Count': 0,
                                        'Played Count': 0,
                                        'Green Card Count': 0,
                                        'Yellow Card Count': 0,
                                        'Red Card Count': 0,
                                        'Starter Count': 0,
                                        'Top Raider Count': 0,
                                        'Top Defender Count': 0,
                                        'Total Points': 0,
                                        'Team ID': team_id,
                                        'Team Name': team_name
                                    }
                                
                                roster[player_id]['Captain Count'] += int(player.get('captain', False))
                                roster[player_id]['Played Count'] += int(player.get('played', False))
                                roster[player_id]['Green Card Count'] += int(player.get('green_card', False))
                                roster[player_id]['Yellow Card Count'] += int(player.get('yellow_card', False))
                                roster[player_id]['Red Card Count'] += int(player.get('red_card', False))
                                roster[player_id]['Starter Count'] += int(player.get('starter', False))
                                roster[player_id]['Top Raider Count'] += int(player.get('top_raider', False))
                                roster[player_id]['Top Defender Count'] += int(player.get('top_defender', False))
                                roster[player_id]['Total Points'] += player.get('points', {}).get('total', 0)

        roster_df = pd.DataFrame(list(roster.values()))
        roster_df['Total Matches in Season'] = total_matches
        return roster_df


    def get_player_info(self, player_id, season=None):
        player_id = int(player_id)
        file_path = "./Player-Wise-Data/all_seasons_player_stats_rounded.csv"
        df = pd.read_csv(file_path)

        file_rvd = Path(r"./Player-Wise-Data/merged_raider_v_num_defenders_FINAL.csv")
        rvd_df = pd.read_csv(file_rvd)

        defend_file = "./Player-Wise-Data/AllSeasons_AllTeams_DefenderSuccessRate.csv"
        defend_df = pd.read_csv(defend_file)

        raider_file = "./Player-Wise-Data/AllSeasons_AllTeams_RaiderSuccessRate.csv"
        raider_df = pd.read_csv(raider_file)

        player_starts = "./Player-Wise-Data/Player_Team_Lineup_merged.csv"
        player_starts_df = pd.read_csv(player_starts)

        def to_numeric_or_nan(x):
            try:
                return pd.to_numeric(x)
            except ValueError:
                return np.nan

        df['player_id'] = df['player_id'].apply(to_numeric_or_nan)
        df['player_id'] = df['player_id'].fillna(-1)
        df['player_id'] = df['player_id'].astype(np.int64)

        rvd_df['player-id'] = rvd_df['player-id-pkdc-sanitised'].apply(to_numeric_or_nan)
        rvd_df['player-id'] = rvd_df['player-id'].fillna(-1)
        rvd_df['player-id'] = rvd_df['player-id'].astype(int)


        defend_df['player_id'] = defend_df['player_id_copy_backup'].apply(to_numeric_or_nan)
        defend_df['player_id'] = defend_df['player_id'].fillna(-1)
        defend_df['player_id'] = defend_df['player_id'].astype(np.int64)

        raider_df['player_id'] = raider_df['player-id-clean'].apply(to_numeric_or_nan)
        raider_df['player_id'] = raider_df['player_id'].fillna(-1)
        raider_df['player_id'] = raider_df['player_id'].astype(np.int64)

        player_starts_df['player_id'] = player_starts_df['player_id_clean'].apply(to_numeric_or_nan)
        player_starts_df['player_id'] = player_starts_df['player_id'].fillna(-1)
        player_starts_df['player_id'] = player_starts_df['player_id'].astype(np.int64)

        # If season is not specified, use the latest season
        if season is None:
            season = df['season'].max()

        # Player aggregated stats
        player_stats_df = df[(df['player_id'] == player_id) & (df['season'] == season)]

        if player_stats_df.empty:
            print(f"No data for player {player_id} for season {season} |  AGGREGATED")

        player_starts_df = player_starts_df[(player_starts_df['player_id'] == player_id) & (player_starts_df['season_num'] == season)]
        if player_starts_df.empty:
            print(f"No data for player {player_id} for season {season} |  STARTS")

        # Raiders v defenders
        rvd_data = rvd_df[rvd_df['player-id'] == player_id]
        if rvd_data.empty:
            print(f"No data for raiders v no of defenders for {player_id}")
        
        rvd_extracted_df = rvd_data[rvd_data['season'].str.extract(r'(\d+)')[0].astype(int) == season]
        if rvd_extracted_df.empty:
            print(f"No data for raiders v no of defenders for {player_id} for season {season}")


        defend_extracted_df = defend_df[(defend_df['player_id'] == player_id) & (defend_df['season'] == season)]
        if defend_extracted_df.empty:
            print(f"No data for defenders for {player_id} for season {season}")
        
        raider_extracted_df = raider_df[(raider_df['player_id'] == player_id) & (raider_df['season'] == season)]
        if raider_extracted_df.empty:
            print(f"No data for raiders for {player_id} for season {season}")
        
        player_stats_df_rank = player_stats_df[["season", "player_id", "player_name", "player_matches_played", "player_position_id", "player_position_name", "team_id", "team_full_name", "player-super-tackles_rank", "player-raid-points_rank", "player-super-raids_rank", "player-high-5s_rank", "player-tackle-points_rank", "player-avg-tackle-points_rank", "player-dod-raid-points_rank", "player-total-points_rank", "player-successful-tackles_rank", "player-successful-raids_rank", "super-10s_rank" ]].copy()
                                                
        player_stats_df_value = player_stats_df[["season", "player_id", "player_name", "player_matches_played", "player_position_id", "player_position_name", "team_id", "team_full_name", "player-super-tackles_value", "player-raid-points_value", "player-super-raids_value", "player-high-5s_value", "player-tackle-points_value", "player-avg-tackle-points_value", "player-dod-raid-points_value", "player-total-points_value", "player-successful-tackles_value", "player-successful-raids_value", "super-10s_value"]].copy()
                                    
        player_stats_df_per_match = player_stats_df[["season", "player_id", "player_name", "player_matches_played", "player_position_id", "player_position_name", "team_id", "team_full_name", "player-super-tackles_points_per_match", "player-raid-points_points_per_match", "player-super-raids_points_per_match", "high-5s_points_per_match", "player-tackle-points_points_per_match", "player-dod-raid-points_points_per_match", "player-total-points_points_per_match", "player-successful-tackles_points_per_match", "player-successful-raids_points_per_match", "super-10s_points_per_match"]].copy()


        if not defend_extracted_df.empty:
            defend_data = defend_extracted_df.iloc[0]
            player_stats_df_value['Total Tackles'] = defend_data.get('Total Tackles', np.nan)
            player_stats_df_value['Successful Tackles'] = defend_data.get('Successful Tackles', np.nan)
            player_stats_df_value['Defender Success Rate'] = defend_data.get('Defender Success rate', np.nan)


        if not raider_extracted_df.empty:
            raider_data = raider_extracted_df.iloc[0]
            player_stats_df_value['Total Raids'] = raider_data.get('Total Raids', np.nan)
            player_stats_df_value['Successful Raids'] = raider_data.get('Successful Raids', np.nan)
            player_stats_df_value['Raider Success Rate'] = raider_data.get('Raider Success Rate', np.nan)

        if not player_starts_df.empty:
            player_starts_data = player_starts_df.iloc[0]
            player_stats_df_value['Total Played'] = player_starts_data.get('Total Played', np.nan)
            player_stats_df_value['Total Starts'] = player_starts_data.get('Total Starts', np.nan)

        rvd_extracted_df = rvd_extracted_df[['Season_Number', 'Team Name',
                                            'player-id', 'Raider Name', 'Number of Defenders', 'Total Raids',
                                            'Percentage of Raids', 'Empty Raids Percentage',
                                            'Successful Raids Percentage']]
        
        

        return player_stats_df_rank.T, player_stats_df_value.T, player_stats_df_per_match.T, rvd_extracted_df.T


    def load_match_details(self, season, match_id) -> Tuple[DataFrame, DataFrame, DataFrame, DataFrame, DataFrame, DataFrame]:
        """
        Get the full data for a specific match.

        Args:
            season (str): The season name.
            match_id (str): The match ID.

        Returns:
            Tuple[DataFrame, DataFrame, DataFrame, DataFrame, DataFrame, DataFrame]:
            A tuple containing match detail, teams, events, zones, team1, and team2 DataFrames.
        """

        # print(f"Loading match details for season {season} and match ID {match_id}")

        for dir in os.listdir(self.base_path):
            if f"Season_PKL_Season_{season}" in dir:
                season_path = os.path.join(self.base_path, dir)
                break
        else:
            raise ValueError(f"No season data found for season {season}")
        
        file_name = next((f for f in os.listdir(season_path) if f.endswith(f'_ID_{match_id}.json')), None)

        if not file_name:
            raise FileNotFoundError(f"No match file found for season {season} and match ID {match_id}")

        file_path = os.path.join(season_path, file_name)

        try:
            with open(file_path, 'r') as file:
                temp = json.load(file)

            # Check if the data is nested under 'gameData'
            if 'gameData' in temp:
                temp = temp['gameData']

            match_detail = temp.get("match_detail", {})
            flattened_match_detail = self._flatten_match_detail(match_detail)
            # print(f"Flattened match detail: {flattened_match_detail}")
            # print(f"Flattened match detail columns: {flattened_match_detail.keys()}")
            # print("-"*100)
            # print(f"Match detail: {match_detail}")
            match_detail_df = pd.DataFrame([flattened_match_detail])

            events_df = pd.DataFrame(temp.get("events", {}).get("event", []))
            zones_df = pd.DataFrame(temp.get("zones", {}).get("zone", []))

            teams_data = temp.get("teams", {}).get("team", [])

            if len(teams_data) != 2:
                raise ValueError("Expected data for exactly two teams")

            team1_df, team2_df = self._process_team_data(teams_data)
            teams_df = pd.DataFrame(teams_data)

            return match_detail_df, teams_df, events_df, zones_df, team1_df, team2_df

        except Exception as e:
            print(f"Error loading data from {file_path}: {str(e)}")
            return None, None, None, None, None, None

    
    def load_pbp_data(self, season, match_id) -> DataFrame:
        """
        Get all events for a specific match.

        Args:
            season (str): The season name.
            match_id (str): The match ID.

        Returns:
            DataFrame: A DataFrame containing all events in the match.
        """
        _, _, events_df, _, _, _ = self.load_match_details(season, match_id)
        return events_df


    def _flatten_match_detail(self, match_detail: Dict[str, Any]) -> Dict[str, Any]:
        """
        Flatten nested dictionaries in match detail.

        Args:
            match_detail (Dict[str, Any]): The match detail dictionary.

        Returns:
            Dict[str, Any]: Flattened match detail dictionary.
        """
        flattened = {}
        for key, value in match_detail.items():
            if isinstance(value, dict):
                for subkey, subvalue in value.items():
                    flattened[f"{key}_{subkey}"] = subvalue

            elif isinstance(value, list):
                if key == "player_of_the_match":
                    flattened[f"{key}_id"] = value[0].get("id") if value else None
                    flattened[f"{key}_value"] = value[0].get("value") if value else None
                else:
                    flattened[key] = json.dumps(value)
            else:
                flattened[key] = value
        return flattened

    def _process_team_data(self, teams_data: List[Dict[str, Any]]) -> Tuple[DataFrame, DataFrame]:
        """
        Process team data into DataFrames.

        Args:
            teams_data (List[Dict[str, Any]]): List containing data for both teams.

        Returns:
            Tuple[DataFrame, DataFrame]: DataFrames for team1 and team2.
        """

        def process_squad(squad_data):
            processed_squad = []
            for player in squad_data:
                player_dict = {
                    'id': player['id'],
                    'name': player['name'],
                    'jersey': player.get('jersey', ''),
                    'played': player.get('played', False),
                    'captain': player.get('captain', False),
                    'on_court': player.get('on_court', False),
                    'starter': player.get('starter', False),
                    'red_card': player.get('red_card', False),
                    'yellow_card': player.get('yellow_card', False),
                    'green_card': player.get('green_card', False),
                    'red_card_count': player.get('red_card_count', 0),
                    'yellow_card_count': player.get('yellow_card_count', 0),
                    'green_card_count': player.get('green_card_count', 0),
                    'top_raider': player.get('top_raider', False),
                    'top_defender': player.get('top_defender', False),
                    'total_points': player.get('points', {}).get('total', 0),
                    'raid_points': player.get('points', {}).get('raid_points', {}).get('total', 0),
                    'tackle_points': player.get('points', {}).get('tackle_points', {}).get('total', 0),
                    'raids_total': player.get('raids', {}).get('total', 0),
                    'raids_successful': player.get('raids', {}).get('successful', 0),
                    'tackles_total': player.get('tackles', {}).get('total', 0),
                    'tackles_successful': player.get('tackles', {}).get('successful', 0),
                }
                for zone_type in ['strong_zones', 'weak_zones']:
                    for zone in player.get(zone_type, {}).get(zone_type.rstrip('s'), []):
                        player_dict[f"{zone_type}_zone_{zone['zone_id']}"] = zone['points']
                processed_squad.append(player_dict)
            return processed_squad

        team1_df = pd.DataFrame(process_squad(teams_data[0].get('squad', [])))
        team2_df = pd.DataFrame(process_squad(teams_data[1].get('squad', [])))

        for team_df, team_data in zip([team1_df, team2_df], teams_data):
            team_df['team_id'] = team_data['id']
            team_df['team_name'] = team_data['name']
            team_df['team_score'] = team_data['score']
            team_df['team_short_name'] = team_data['short_name']

        return team1_df, team2_df










# Instantiating the class



In [2]:

api = KabaddiDataAPI()


# Standings Data

The first point from which analysis can be done is the standings data. This function returns the standings data for a given season. This enables us to see zoomed out stats like how many matches each team has played, how many matches they have won or lost, and the points they have scored. This can inspire us to investigate a particular team futher


In [35]:

qualified_df , all_standings_df = api.get_pkl_standings_matches(season=9, qualified=True)
print("qualified_df")
qualified_df


# print("all_standings_df")
# print(all_standings_df)
# print(len(all_standings_df))


qualified_df


,Group,Season,Team_Id,Team_Name,League_position,Matches_played,Wins,Lost,Tied,Draws,No Result,League_points,Score_diff,Qualified
0,Main,9,3,Jaipur Pink Panthers,1,22,15,6,1,0,0,82,174,True
1,Main,9,7,Puneri Paltan,2,22,14,6,2,0,0,80,66,True
2,Main,9,1,Bengaluru Bulls,3,22,13,8,1,0,0,74,39,True
3,Main,9,30,UP Yoddhas,4,22,12,8,2,0,0,71,42,True
4,Main,9,29,Tamil Thalaivas,5,22,10,8,4,0,0,66,5,True
5,Main,9,2,Dabang Delhi K.C.,6,22,10,10,2,0,0,63,17,True


# Season Matches

The second point of data that can be analysed is the season matches. This function returns all the matches that have been played in a season. This can be used to see the schedule of the season, or to see the matches that a particular team has played. This can inspire us to investigate a particular team further...


In [36]:

print("season_matches")
matches = api.get_season_matches(season=6)
matches
# print(len(x))



season_matches


,Season,Match_ID,Match_Name,League_Stage,Year,Venue,Match_Outcome,Start_Date,End_Date,Result,Winning Margin,team_score_1,team_score_2,team_name_1,team_id_1,team_name_2,team_id_2
0,6,625,Match 1,League,2018,"Jawaharlal Nehru Indoor Stadium, Chennai",Tamil Thalaivas Won by 16 Pts,2018-10-07T20:00+05:30,2018-10-07T20:00+05:30,W,16,42,26,Tamil Thalaivas,29,Patna Pirates,6
1,6,626,Match 2,League,2018,"Jawaharlal Nehru Indoor Stadium, Chennai",,2018-10-07T21:00+05:30,2018-10-07T21:00+05:30,Tied,,32,32,Puneri Paltan,7,U Mumba,5
2,6,627,Match 3,League,2018,"Jawaharlal Nehru Indoor Stadium, Chennai",Puneri Paltan Won by 12 Pts,2018-10-08T20:00+05:30,2018-10-08T20:00+05:30,W,12,34,22,Puneri Paltan,7,Haryana Steelers,28
3,6,628,Match 4,League,2018,"Jawaharlal Nehru Indoor Stadium, Chennai",U.P. Yoddha Won by 5 Pts,2018-10-08T21:00+05:30,2018-10-08T21:00+05:30,W,5,32,37,Tamil Thalaivas,29,U.P. Yoddha,30
4,6,629,Match 5,League,2018,"Jawaharlal Nehru Indoor Stadium, Chennai",,2018-10-09T20:00+05:30,2018-10-09T20:00+05:30,Tied,,32,32,Dabang Delhi K.C.,2,Gujarat Giants,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,6,758,Match 134,Playoffs,2018,"Rajiv Gandhi Indoor Stadium, Kochi",Dabang Delhi K.C. Won by 11 Pts,2018-12-30T21:00+05:30,2018-12-30T21:00+05:30,W,11,39,28,Dabang Delhi K.C.,2,Bengal Warriors,4
134,6,759,Match 135,Playoffs,2018,"Rajiv Gandhi Indoor Stadium, Kochi",Bengaluru Bulls Won by 12 Pts,2018-12-31T20:00+05:30,2018-12-31T20:00+05:30,W,12,29,41,Gujarat Giants,31,Bengaluru Bulls,1
135,6,760,Match 136,Playoffs,2018,"Rajiv Gandhi Indoor Stadium, Kochi",U.P. Yoddha Won by 12 Pts,2018-12-31T21:00+05:30,2018-12-31T21:00+05:30,W,12,45,33,U.P. Yoddha,30,Dabang Delhi K.C.,2
136,6,761,Match 137,Playoffs,2018,"Dome,Nsci,Svp Stadium,Mumbai, Mumbai",Gujarat Giants Won by 7 Pts,2019-01-03T20:00+05:30,2019-01-03T20:00+05:30,W,7,38,31,Gujarat Giants,31,U.P. Yoddha,30


# Team Info

The third point of data that can be analysed is the team info. This function returns the team info for a given season and team id. This can be used to see the stats of a particular team. This can inspire us to investigate the team further or also to compare the stats of multiple teams. 

Additinoally, this function also returns the raider skills and defender skills for a given team. This can inspire us to investigate the team further or also to compare the skills of multiple teams.
    


In [38]:

# print("-"*100)
print("team_info")

df_rank_, df_value_, df_per_match_, filtered_team_raider_skills_, filtered_team_defender_skills_ = api.get_team_info(season=6,team_id=29)

print("Rank DFs")
df_rank_



team_info
Rank DFs


,53
season,6.0
team_id,29
team_name,Tamil Thalaivas
matches_played,22
team-average-raid-points_rank,7
team-avg-points-scored_rank,10
team-all-outs-conceded_rank,7
team-successful-tackle-percent_rank,NaN
team-super-raid_rank,11
team-raid_rank,NaN


In [39]:

print("Value DFs")
df_value_


Value DFs


,53
season,6.0
team_id,29
team_name,Tamil Thalaivas
matches_played,22
team-all-outs-conceded_value,27
team-successful-tackle-percent_value,NaN
team-super-raid_value,7
team-raid_value,NaN
team-successful-raid-percent_value,NaN
team-dod-raid-points_value,67


In [40]:

print("Per Match DFs")
df_per_match_


Per Match DFs


,53
season,6.0
team_id,29
team_name,Tamil Thalaivas
matches_played,22
team-all-outs-conceded_per-match,1.227
team-super-raid_per-match,0.318
team-raid_per-match,0.0
team-dod-raid-points_per-match,3.045
team-super-tackles_per-match,0.682
team-total-touch-points-ext_per-match,14.318


In [41]:

print("Raider Skills")
filtered_team_raider_skills_


Raider Skills


,Season,Skill Type,Skill Name,Tamil Thalaivas (29)
0,6,Counter Action Skill,Release,49.0
1,6,Counter Action Skill,Struggle,17.0
2,6,Counter Action Skill,Out Turn,19.0
3,6,Counter Action Skill,Jump,17.0
4,6,Counter Action Skill,In Turn,9.0
5,6,Counter Action Skill,Create Gap,9.0
6,6,Counter Action Skill,Dubki,3.0
7,6,Attacking Skill,Running Hand Touch,91.0
8,6,Attacking Skill,Hand Touch,56.0
9,6,Attacking Skill,Leg Thrust,14.0


In [42]:


print("Defender Skills")
filtered_team_defender_skills_


#TODO: Fix the order of the columns in the output
# team-successful-tackle-percent_value
# team-super-tackles_value
# team-tackle-points_value
# team-successful-tackles-per-match_value
# team-average-tackle-points_value
# team-successful-tackles_value
# Total_touch_points_value
# Total_bonus_points_value
# team-raid-points_value
# team-average-raid-points_value
# team-successful-raids_value
# team-super-raid_value
# team-raid_value
# team-successful-raid-percent_value
# team-dod-raid-points_value
# team-total-points_value
# team-all-outs-conceded_value
# team-all-outs-inflicted_value
# team-avg-points-scored_value
# team-total-points-conceded_value


Defender Skills


,Season,Skill Type,Skill Name,Tamil Thalaivas (29)
0,6,Defensive Skill,Block,47.0
1,6,Defensive Skill,Dive,30.0
2,6,Defensive Skill,Thigh Hold,24.0
3,6,Defensive Skill,Ankle Hold,16.0
4,6,Defensive Skill,Body Hold,22.0
5,6,Defensive Skill,Push,27.0
6,6,Defensive Skill,Follow,NaN
7,6,Defensive Skill,Self-out,3.0
8,6,Counter Action Skill,Release,81.0
9,6,Counter Action Skill,Struggle,13.0


# Get-Team Matches

The next function that can be analysed is the get_team_matches function. This function returns all the matches that a particular team has played in a given season. This can be used to see the schedule of the matches that a particular team has played. This can inspire us to investigate a particular team further...


In [43]:



print("get_team_matches\n\n")

df = api.get_team_matches(season=9, team_id=3)
df



get_team_matches




,Season,Match_ID,Match_Name,League_Stage,Year,Venue,Match_Outcome,Start_Date,End_Date,Result,Winning Margin,team_score_1,team_score_2,team_name_1,team_id_1,team_name_2,team_id_2
2,9,2894,Match 3,League,2022,"Shree Kanteerava Indoor Stadium, Bengaluru",U.P. Yoddhas won by 2 Pts,2022-10-07T21:30+05:30,2022-10-07T21:30+05:30,W,2,32,34,Jaipur Pink Panthers,3,U.P. Yoddhas,30
6,9,2898,Match 7,League,2022,"Shree Kanteerava Indoor Stadium, Bengaluru",Jaipur Pink Panthers won by 5 Pts,2022-10-09T19:30+05:30,2022-10-09T19:30+05:30,W,5,35,30,Jaipur Pink Panthers,3,Patna Pirates,6
16,9,2908,Match 17,League,2022,"Shree Kanteerava Indoor Stadium, Bengaluru",Jaipur Pink Panthers won by 13 Pts,2022-10-14T20:30+05:30,2022-10-14T20:30+05:30,W,13,31,44,Haryana Steelers,28,Jaipur Pink Panthers,3
18,9,2910,Match 19,League,2022,"Shree Kanteerava Indoor Stadium, Bengaluru",Jaipur Pink Panthers won by 7 Pts,2022-10-15T19:30+05:30,2022-10-15T19:30+05:30,W,7,25,18,Jaipur Pink Panthers,3,Gujarat Giants,31
25,9,2917,Match 26,League,2022,"Shree Kanteerava Indoor Stadium, Bengaluru",Jaipur Pink Panthers won by 15 Pts,2022-10-18T19:30+05:30,2022-10-18T19:30+05:30,W,15,24,39,Bengal Warriors,4,Jaipur Pink Panthers,3
33,9,2925,Match 34,League,2022,"Shree Kanteerava Indoor Stadium, Bengaluru",Jaipur Pink Panthers won by 24 Pts,2022-10-22T20:30+05:30,2022-10-22T20:30+05:30,W,24,51,27,Jaipur Pink Panthers,3,Telugu Titans,8
37,9,2929,Match 38,League,2022,"Shree Kanteerava Indoor Stadium, Bengaluru",Puneri Paltan won by 8 Pts,2022-10-25T19:30+05:30,2022-10-25T19:30+05:30,W,8,32,24,Puneri Paltan,7,Jaipur Pink Panthers,3
41,9,2933,Match 42,League,2022,"Shree Shivchhatrapati Sports Complex,Balewadi,...",Tamil Thalaivas won by 11 Pts,2022-10-28T19:30+05:30,2022-10-28T19:30+05:30,W,11,38,27,Tamil Thalaivas,29,Jaipur Pink Panthers,3
47,9,2939,Match 48,League,2022,"Shree Shivchhatrapati Sports Complex,Balewadi,...",Bengaluru Bulls won by 6 Pts,2022-10-30T19:30+05:30,2022-10-30T19:30+05:30,W,6,31,37,Jaipur Pink Panthers,3,Bengaluru Bulls,1
56,9,2948,Match 57,League,2022,"Shree Shivchhatrapati Sports Complex,Balewadi,...",Jaipur Pink Panthers won by 5 Pts,2022-11-04T20:30+05:30,2022-11-04T20:30+05:30,W,5,40,45,Dabang Delhi K.C.,2,Jaipur Pink Panthers,3


# Build Team Roster

The next function that can be analysed is the build_team_roster function. This function returns the roster of a particular team in a given season. This can be used to see the players that are in a particular team in a given season. This can inspire us to investigate a particular player further with the player_id being mentioned and acting as a parameter for the function `get_player_info`.


In [44]:



print("build build_team_roster")

df = api.build_team_roster(season=9, team_id='3')
df

## df.to_csv("team_roster.csv")


build build_team_roster


,Player ID,Name,Jersey Number,Captain Count,Played Count,Green Card Count,Yellow Card Count,Red Card Count,Starter Count,Top Raider Count,Top Defender Count,Total Points,Team ID,Team Name,Total Matches in Season
0,2024,Arjun Deshwal,2,0,24,1,0,0,23,20,0,296,3,Jaipur Pink Panthers,24
1,368,Sunil Kumar,11,23,23,1,0,0,23,0,5,65,3,Jaipur Pink Panthers,24
2,5022,Reza Mirbagheri,1,0,16,0,0,0,13,0,1,36,3,Jaipur Pink Panthers,24
3,81,Rahul Chaudhari,9,1,21,0,0,0,20,1,0,73,3,Jaipur Pink Panthers,24
4,3053,V Ajith Kumar,6,0,21,0,0,0,20,2,0,103,3,Jaipur Pink Panthers,24
5,4769,Sahul Kumar,66,0,24,2,0,0,23,0,4,57,3,Jaipur Pink Panthers,24
6,3574,Ankush,3,0,24,2,0,0,24,0,12,89,3,Jaipur Pink Panthers,24
7,5113,Abhishek KS,4,0,18,0,0,0,17,0,1,25,3,Jaipur Pink Panthers,24
8,4956,Deepak Singh,5,0,3,0,0,0,1,0,0,2,3,Jaipur Pink Panthers,24
9,5042,Lucky Sharma,8,0,15,1,0,0,0,0,1,11,3,Jaipur Pink Panthers,24


# Get Player Info

The next function that can be analysed is the `get_player_info` function. This function returns the player info for a given player_id and season. This can be used to see the aggregated stats of a particular player over the entire season. Can be used to compare the stats of multiple players. Will be helpful in building a player info dataframe which can be used to build a player info dashboard.


We notabley return a raider vs defenders dataframe which can be used to analyse the skills of a raider vs the number of defenders on the opponent team. This is a very important skill in kabaddi and can be used to win or lose matches.



In [50]:


print("build get_player_info")

_player_stats_df_rank, _player_stats_df_value, _player_stats_df_per_match, _rvd_extracted_df = api.get_player_info(player_id=660, season=9)

_player_stats_df_rank


build get_player_info


,761
season,9
player_id,660
player_name,Bhavani Rajput
player_matches_played,14
player_position_id,8
player_position_name,Raider
team_id,3
team_full_name,Jaipur Pink Panthers
player-super-tackles_rank,NaN
player-raid-points_rank,43.0


In [51]:
_player_stats_df_value


,761
season,9
player_id,660
player_name,Bhavani Rajput
player_matches_played,14
player_position_id,8
player_position_name,Raider
team_id,3
team_full_name,Jaipur Pink Panthers
player-super-tackles_value,NaN
player-raid-points_value,36.0


In [52]:
_player_stats_df_per_match


,761
season,9
player_id,660
player_name,Bhavani Rajput
player_matches_played,14
player_position_id,8
player_position_name,Raider
team_id,3
team_full_name,Jaipur Pink Panthers
player-super-tackles_points_per_match,NaN
player-raid-points_points_per_match,2.57


In [53]:

_rvd_extracted_df



,2169,2170,2171,2172,2173,2174
Season_Number,9,9,9,9,9,9
Team Name,Jaipur Pink Panthers,Jaipur Pink Panthers,Jaipur Pink Panthers,Jaipur Pink Panthers,Jaipur Pink Panthers,Jaipur Pink Panthers
player-id,660,660,660,660,660,660
Raider Name,Bhavani Rajput,Bhavani Rajput,Bhavani Rajput,Bhavani Rajput,Bhavani Rajput,Bhavani Rajput
Number of Defenders,2,3,4,5,6,7
Total Raids,3,4,12,15,11,32
Percentage of Raids,4.00%,5.00%,16.00%,19.00%,14.00%,42.00%
Empty Raids Percentage,66.70%,50.00%,16.70%,40.00%,45.50%,34.40%
Successful Raids Percentage,33.30%,50.00%,50.00%,33.30%,18.20%,37.50%


# Load Match Details

The next function that can be analysed is the `load_match_details` function. This function returns the match details for a given season and match_id. This can be used to see the match details for a particular match. This can inspire us to investigate a particular match further...
    
This is granular match detail which explains the events, teams, zones and the points scored in the match in a granular and chronological manner.






In [54]:

print("load_match_details-------\n\n\n")
_match_detail_df, _teams_df, _events_df, _zones_df, _team1_df, _team2_df = api.load_match_details(season='9', match_id='2897')
print("-"*100)
_match_detail_df.T


load_match_details-------



----------------------------------------------------------------------------------------------------


,0
match_id,2897
match_number,Match 6
clock_minutes,40
clock_seconds,1
date,10/8/2022
start_time,21:30
matchtime_iso,2022-10-08T16:00:00Z
gmtoffset,+05:30
result_outcome,None
result_value,Haryana Steelers Beat Bengal Warriors (41-33 )


In [55]:

# print("-"*100)
_teams_df


,id,name,score,short_name,squad,extra_time,golden_raid,stats,state_of_play,green_card_count,yellow_card_count,red_card_count
0,4,Bengal Warriors,33,Bengal,"[{'id': 143, 'name': 'Maninder Singh', 'short_...",{'score': ''},{'score': ''},"{'points': {'total': 33, 'raid_points': {'tota...","{'is_raiding_now': False, 'players_on_court': ...",1,0,0
1,28,Haryana Steelers,41,Haryana,"[{'id': 4183, 'name': 'Meetu', 'short_name': '...",{'score': ''},{'score': ''},"{'points': {'total': 41, 'raid_points': {'tota...","{'is_raiding_now': False, 'players_on_court': ...",0,0,0


In [56]:

# print("-"*100)
_events_df


,event_no,event_half,event,event_id,event_text,raider_id,raiding_team_id,defender_id,defending_team_id,raid_points,...,clock,status_id,score,seq_no,defenders,created_date,player_id,substituted_by,team_id,substitute_time
0,1,1,Unsuccessful Raid,2,Maninder Singh unsuccessful raid,143.0,4.0,4954.0,28.0,0.0,...,20:00,0,"[0, 1]",1,[4954],2022-10-08T21:51:22,NaN,NaN,NaN,NaN
1,2,1,Unsuccessful Raid,2,Meetu unsuccessful raid,4183.0,28.0,161.0,4.0,0.0,...,19:21,0,"[1, 1]",2,[161],2022-10-08T21:51:45,NaN,NaN,NaN,NaN
2,3,1,Unsuccessful Raid,2,Deepak Hooda unsuccessful raid,41.0,4.0,5101.0,28.0,0.0,...,18:59,0,"[1, 2]",3,[5101],2022-10-08T21:52:12,NaN,NaN,NaN,NaN
3,4,1,Successful Raid,1,Manjeet raids successfully,763.0,28.0,NaN,4.0,2.0,...,18:32,0,"[1, 4]",4,[3103],2022-10-08T21:52:47,NaN,NaN,NaN,NaN
4,5,1,Empty Raid,3,Maninder Singh empty raid,143.0,4.0,NaN,28.0,0.0,...,18:01,0,"[1, 4]",5,[],2022-10-08T21:53:20,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,101,2,Successful Raid,1,Manjeet raids successfully,763.0,28.0,NaN,4.0,3.0,...,00:14,0,"[31, 41]",84,"[161, 202, 3103]",2022-10-08T22:52:04,NaN,NaN,NaN,NaN
100,103,2,Substitution,5,Sunny comes in for Mohit,NaN,NaN,NaN,NaN,NaN,...,00:02,0,None,0,NaN,2022-10-08T22:52:04,4954.0,5104.0,28.0,40.0
101,104,2,Substitution,5,Harsh comes in for Jaideep,NaN,NaN,NaN,NaN,NaN,...,00:02,0,None,0,NaN,2022-10-08T22:52:04,4184.0,5103.0,28.0,40.0
102,102,2,Timeout,8,None,NaN,NaN,NaN,NaN,NaN,...,00:02,0,None,0,NaN,2022-10-08T22:52:04,NaN,NaN,28.0,NaN


In [57]:

# print("-"*100)
_zones_df


,id,name
0,1,Left Lobby
1,2,Right Lobby
2,3,Midline Left
3,4,Midline Centre
4,5,Midline Right
5,6,Baulk Left
6,7,Baulk Centre
7,8,Baulk Right
8,9,Bonus Left
9,10,Bonus Centre


In [58]:

# print("-"*100)
_team1_df


,id,name,jersey,played,captain,on_court,starter,red_card,yellow_card,green_card,...,weak_zones_zone_6,weak_zones_zone_7,weak_zones_zone_8,weak_zones_zone_9,weak_zones_zone_10,weak_zones_zone_11,team_id,team_name,team_score,team_short_name
0,143,Maninder Singh,9,True,True,True,True,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
1,202,Ashish Sangwan,10,True,False,True,True,False,False,True,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
2,4978,Balaji D,2,True,False,True,True,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
3,41,Deepak Hooda,5,True,False,False,True,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
4,106,Shrikant Jadhav,22,True,False,True,True,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
5,3103,Shubham Shinde,69,True,False,False,True,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
6,161,Girish Maruti Ernak,8,True,False,True,True,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
7,146,Surender Nada,1,False,False,False,False,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
8,4675,Akash Pikalmunde,4,False,False,False,False,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal
9,5114,Vaibhav Garje,15,True,False,False,False,False,False,False,...,0,0,0,0,0,0,4,Bengal Warriors,33,Bengal


In [59]:

# print("-"*100)
_team2_df.T



,0,1,2,3,4,5,6,7,8,9,10,11,12
id,4183,4954,4184,763,3065,5101,194,5103,204,4977,5104,219,3054
name,Meetu,Mohit,Jaideep Dahiya,Manjeet,Nitin Rawal,Monu,Joginder Narwal,Harsh,Rakesh Narwal,Ankit,Sunny,K. Prapanjan,Vinay
jersey,2,7,5,3,44,99,8,4,9,12,15,25,33
played,True,True,True,True,True,True,True,True,True,True,True,False,True
captain,False,False,False,False,False,False,True,False,False,False,False,False,False
on_court,True,False,False,True,False,True,True,False,False,False,False,False,False
starter,True,True,True,True,True,True,True,False,False,False,False,False,False
red_card,False,False,False,False,False,False,False,False,False,False,False,False,False
yellow_card,False,False,False,False,False,False,False,False,False,False,False,False,False
green_card,False,False,False,False,False,False,False,False,False,False,False,False,False


# Load PBP Data

The next function that can be analysed is the `load_pbp_data` function. This function returns the pbp data for a given season and match_id. This can be used to see the pbp data for a particular match. 


In [61]:

print("load_pbp_data")
_pbp_df = api.load_pbp_data(season=9, match_id='2895')
print("-"*100)
_pbp_df























# print(api.get_available_seasons())


# match_detail_df, teams_df, events_df, zones_df, team1_df, team2_df = api.get_match_data('Season_PKL_Season_4_2016',
#                                                                                         '194')
# print(events_df)






# x = api.build_team_roster('4',9)
# print(x)
# x = api.get_team_matches('5','4')
# print(x)

# x = api.get_season_matches('10')
# print(x)

# x = api.get_team_info(4)
# print(x)

# player_stats, rvd,defend_df= api.get_player_info(322,9)
# print(player_stats)
# print(rvd)
# print(defend_df)


# x = api.get_pkl_standings(season=7, qualified=False)
# print(x)
# # print("-"*100)
# # print(y)
###########################################
# print("-" * 100, "test--", "-" * 100)
#
# x, y = api.get_pkl_standings(season=1, qualified=True)
# print(x)
# print("-" * 100)
# print(y)
#
# print("-" * 100)

# print(x.to_latex())

# print(m)

# df = api.match_data(season=4)
# print(df)

# for a season -> display all the match ids for that season along with some basic info about the match (matches overview)
# for a given match-id -> get play-by-play data

# for a given team in a given season return aggregated stats for that team
# for a given team in a season, return the defender skills and raider skills for that team

# return all matches played by that team

# get team roster with player-ids
# for a given player-id return aggregated stats about that player


load_pbp_data
----------------------------------------------------------------------------------------------------


,event_no,event_half,event,event_id,event_text,raider_id,raiding_team_id,defender_id,defending_team_id,raid_points,...,clock,status_id,score,seq_no,defenders,created_date,player_id,substituted_by,team_id,substitute_time
0,1,1,Successful Raid,1,Aslam Inamdar raids successfully,4960.0,7.0,NaN,6.0,1.0,...,19:59,0,"[0, 1]",1,[],2022-10-08T19:33:27,NaN,NaN,NaN,NaN
1,2,1,Empty Raid,3,Sachin empty raid,757.0,6.0,NaN,7.0,0.0,...,19:30,0,"[0, 1]",2,[],2022-10-08T19:33:47,NaN,NaN,NaN,NaN
2,3,1,Successful Raid,1,Mohit Goyat raids successfully,4022.0,7.0,NaN,6.0,1.0,...,19:09,0,"[0, 3]",3,"[3107, 5092]",2022-10-08T19:34:28,NaN,NaN,NaN,NaN
3,4,1,Successful Raid,1,Vishwas S raids successfully,4757.0,6.0,NaN,7.0,2.0,...,18:25,0,"[2, 3]",4,"[4022, 3011]",2022-10-08T19:35:01,NaN,NaN,NaN,NaN
4,5,1,Successful Raid,1,Akash Shinde raids successfully,4959.0,7.0,NaN,6.0,1.0,...,17:52,0,"[2, 4]",5,[3023],2022-10-08T19:35:25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,2,Substitution,5,Aditya Shinde comes in for Balasaheb Jadhav,NaN,NaN,NaN,NaN,NaN,...,00:57,0,None,0,NaN,2022-10-08T20:28:11,3011.0,5116.0,7.0,40.0
93,93,2,Timeout,8,None,NaN,NaN,NaN,NaN,NaN,...,00:57,0,None,0,NaN,2022-10-08T20:28:11,NaN,NaN,6.0,NaN
94,95,2,Empty Raid,3,Rohit Gulia empty raid,3023.0,6.0,NaN,7.0,0.0,...,00:57,0,"[34, 34]",84,[],2022-10-08T20:30:08,NaN,NaN,NaN,NaN
95,96,2,Timeout,8,None,NaN,NaN,NaN,NaN,NaN,...,00:28,0,None,0,NaN,2022-10-08T20:30:08,NaN,NaN,7.0,NaN
